# Feedthrough Randomizer
A feedthrough randomizer is a modulo 2 IIR filter descibed by the difference equation

\begin{equation*}
y_{ran}(n) = x(n) + \sum_{k=1}^{M} h_k y(n-k)
\end{equation*}

where the coefficients $h_k$ are the binary values of the connection polynomial.
So for a $R(0,2,3)$ connection polynomial, $h=[1,0,1,1]$.  This performs
the polynomial division of $x$ by $h$.  The follwing class implements the feedthrough randomizer difference equation:



In [1]:
import numpy as np

class rand(object):

    def __init__(self, h, state=None):
        ''' Implements feed-through randomizer.

        Args:
            h (array): Randomizer taps.  For R(0,2,3) polynmial, h = [1,0,1,1].
        '''

        self.h = h
        if state is None:
            self.state = np.zeros(len(h)-1)
        else:
            self.state = state

    def run(self, x):
        ''' Randomize block of data.'''

        y = np.zeros(len(x))

        for n in range(len(x)):
            mask = np.logical_and(self.state, self.h[1:])
            y[n] = (x[n] + mask.sum()) % 2
            self.state[1:] = self.state[0:-1]
            self.state[0] = y[n]

        return y

To calculate impulse response of randomizer:

In [2]:
h = [1,0,1,1]
x = [0]*16
x[0] = 1
r = rand(h)
r.run(x)

array([1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0.])

Note that the impulse response of the feedthrough radomizer is a $`R(0,2,3)`$ LRS.

# Feedthrough Derandomizer
A feedtrough (or self-synchronizing) derandomizer is a modulo 2 FIR filter
descibed by the difference equation

\begin{equation*}
y_{der}(n) = \sum_{k=0}^{M} h_k x(n-k)
\end{equation*}

where the coefficients $h_k$ are the binary values of the connection polynomial.
So for a $R(0,2,3)$ connection polynomial, $h=[1,0,1,1]$.  This performs
the polynomial multiplication of $h$ and $x$.  The follwing class implements the feedthrough derandomizer difference equation:

In [3]:
class derand(object):

    def __init__(self, h):
        ''' Implements feed-through derandomizer.

        Args:
            h (array): Derandomizer taps.  For R(0,2,3) polynmial, h = [1,0,1,1].
        '''

        self.h = h
        self.state = np.zeros(len(h))

    def run(self, x):
        ''' Derandomize block of data.'''

        y = np.zeros(len(x))

        for n in range(len(x)):
            self.state[1:] = self.state[0:-1]
            self.state[0] = x[n]
            mask = np.logical_and(self.state, self.h)
            y[n] = mask.sum() % 2

        return y

Calculate impulse response of derandomizer:

In [4]:
h = [1,0,1,1]
x = [0]*16
x[0] = 1
d = derand(h)
d.run(x)

array([1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

# Using Feedthrough Derandomizer to Detect Bit Errors in PN Sequence
If the input $x(n)$ to a feedthrough randomizer is zero, the difference equation becomes

\begin{equation*}
y_{ran}(n) = \sum_{k=1}^{M} h_k y(n-k)
\end{equation*}

If the initial state of the randomizer is not zero, it will generate and LRS according to its taps.

If we apply the $Z$ transform to the difference equations of the feedthrough randomizer and derandomizer and determine their respective transfer functions

\begin{equation*}
Y_{der}(z) = \sum_{k=0}^{M} h_k X(z)z^{-k} \Longrightarrow H_{der}(Z) = \sum_{k=0}^{M} h_k z^{-k}
\end{equation*}

\begin{equation*}
Y_{ran}(z) = X(Z) + \sum_{k=1}^{M} h_k Y(z)z^{-k} \Longrightarrow H_{ran}(Z) = \frac{1}{1 - \sum_{k=1}^{M} h_k z^{-k}}
\end{equation*}

we can show that the derandomizer transfer function cancels the randomizer transfer function,

\begin{equation*}
Y(z) = X(z) H_{ran}(z) H_{der}(z) = 
X(z)\left\{\sum_{k=0}^M h_k z^{-k} \right\} \left\{\frac{1}{1 - \sum_{k=1}^{M} h_k z^{-k}} \right\} =
X(z)\left\{\frac{\sum_{k=0}^{M} h_k z^{-k}}{1 - \sum_{k=1}^{M} h_k z^{-k}} \right\} =
X(z)\left\{\frac{h_0 + \sum_{k=1}^{M} h_k z^{-k}} {1 - \sum_{k=1}^{M} h_k^{-k}} \right\} = 
X(z) h_0 = X(z)
\end{equation*}

Therefore, we can use a feedthrough derandomizer to detect bit errors in a PN sequence by setting its taps to the connection polynomial of the PN sequence generator.  Note that
the derandomizer output will have a transient of  $len(h)-1$ bits.

The following Python code demonstrates this.

In [5]:
# Taps
h = [1,0,1,1]
# Zero input to randomizer
x = [0]*16
# Randomizer initial state
state = [1,0,0]
# Create randomizer object
r = rand(h, state)
# Randomize input
y_ran = r.run(x)
y_ran

array([0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1.])

In [6]:
# Annihilate PN sequence
d = derand(h)
y_der = d.run(y_ran)
# Remove transient
transient = len(h) - 1
y_der[transient:]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

The following Python shows the detection of a singel bit error in a PN sequence.  Note that as the bit error propates though the derandomizer's delay line, an error is indicated for each coefficient in the connection polynomial.  Therefore, to use this technique to measure bit errors the total number of $`1`$'s seen in output stream needs to be scaled by the number of non-zero taps in coefficient polynomial.

In [7]:
# Taps
h = [1,0,1,1]

# Zero input to randomizer
x = [0]*16

# Randomizer initial state
state = [1,0,0]

# Create randomizer object
r = rand(h, state)

# Randomize input
y_ran = r.run(x)

# Insert bit error
y_ran[5] = not(y_ran[5])

# Annihilate PN sequence 
d = derand(h)
y_der = d.run(y_ran)

# Remove transient
transient = len(h) - 1
y_der[transient:]

# Calculate bit errors
num_errs = y_der[transient:].sum()/(len(h)-1)
num_errs

1.0